In [10]:
import json
import numpy as np

test_ids = json.load(open("./data/test_ids.json"))
id_meta_map = json.load(open("./data/cloth_body_face_meta.json"))

In [11]:
def get_cat_attr_mask(cat_code, attr_code):
    cat_attr_mask = np.zeros((len(cat_code), len(attr_code)), dtype=np.float)
    cat_attr_map = json.load(open("./data/clothes_category_attribute_value.json"))
    for cat, res in cat_attr_map.items():
        if cat not in cat_code:
            continue
        tmp_cat_code = cat_code[cat]
        for each in res:
            for attr, vals in each.items():
                if attr not in attr_code:
                    continue
                tmp_attr_code = attr_code[attr]
                cat_attr_mask[tmp_cat_code][tmp_attr_code] = 1

    return cat_attr_mask

cat_attr_mask = get_cat_attr_mask(cat_code, attr_code)

In [12]:
cat_code = json.load(open("./data/code_cat.json"))
attr_code = json.load(open("./data/code_attr.json"))
attr_val_code = json.load(open("./data/code_attr_val.json"))
occ_code = json.load(open("./data/code_occasion.json"))
cat_attr = json.load(open("./data/clothes_category_attribute_value.json"))

def rev(k_v):
    res = {}
    for k, v in k_v.items():
        res[v] = k
    return res

code_occ = rev(occ_code)
code_cat = rev(cat_code)
code_attr = rev(attr_code)
attr_code_val = {}
for attr, res in attr_val_code.items():
    attr_code_val[attr] = rev(res)

In [28]:
def recover(attr_res, cat_res, cat_mask, occ_res, cat_attr_mask):   
    occ = code_occ[occ_res]
    cats = []
    attrs = []
    for i, m in enumerate(cat_mask.tolist()):
        if m != 1:
            break
        else:
            cat = code_cat[cat_res[i]]
            cats.append(cat)
            tmp_attrs = []
            for j, mask in enumerate(cat_attr_mask[cat_res[i]].tolist()):
                if mask == 1:
                    tmp_attr = code_attr[j]
                    val_code = attr_res[i][j]
                    attr = "%s:%s" %(tmp_attr, attr_code_val[tmp_attr][val_code])
                    tmp_attrs.append(attr)
            attrs.append(tmp_attrs)
                    
    return occ, cats, attrs

In [29]:
attr_res = np.load("./result/attr_val_res.npy")
cat_res = np.load("./result/cat_res.npy")
occ_res = np.load("./result/occ_res.npy")
cat_mask = np.load("./result/cat_mask.npy")

In [37]:
print(attr_res.shape)
print(len(test_ids))

(8060, 5, 8)
8062


In [42]:
import copy

def generate_test_file(id_meta_map, test_ids, attr_res, cat_res, occ_res, cat_mask, cat_attr_mask):
    test_res = {}
    for i, img_id in enumerate(test_ids[0:len(attr_res)]):
        tmp = id_meta_map[img_id]
        occ, cats, attrs = recover(attr_res[i], cat_res[i], cat_mask[i], occ_res[i], cat_attr_mask)
        
        assert len(cats) == len(attrs)
        assert len(cats) == len(id_meta_map[img_id]["cloth_body_face"])
        
        tmp_cont = copy.deepcopy(id_meta_map[img_id])
        
        for j, cloth in enumerate(id_meta_map[img_id]["cloth_body_face"]):
            attr_val = {}
            for each in attrs[j]:
                each_attr, each_val = each.split(":")
                attr_val[each_attr] = each_val
            
            cloth = cloth["cloth"]
            for k, v in cloth.items():
                if k == "category":
                    tmp_cont["cloth_body_face"][j]["cloth"]["category"] = cats[j]
                elif k == "box":
                    continue
                elif ":" in k:
                    attr, val = k.split(":")
                    if attr not in attr_val:
                        del tmp_cont["cloth_body_face"][j]["cloth"][k]
                        continue
                    if val == attr_val[attr]:
                        continue
                    else:
                        del tmp_cont["cloth_body_face"][j]["cloth"][k]
                        k = ":".join([attr, attr_val[attr]])
                        tmp_cont["cloth_body_face"][j]["cloth"][k] = 123 
        test_res[img_id] = tmp_cont
                        
    return test_res

test_res = generate_test_file(id_meta_map, test_ids, attr_res, cat_res, occ_res, cat_mask, cat_attr_mask)

In [51]:
json.dump(test_res, open("./result/test_cloth_body_meta.json", "w"))

In [46]:
print(len(test_res))
for i in range(10):
    print(test_ids[i])

8060
369d3e848144c3052f6480824f434b71
1428da85ef675ff042a2c4e8211ff90c
64c54e53c153e17fbcb608191b996103
72b3b4cf06c0315e82e4be3799b326fa
142eb3089b93c4d8eb27f97b323eb259
1d59fec94ab3e2c9f799397ae459e7e9
5350fc4a5a7904228d4bead20258a6ff
6997532374da952ee6c8c98f8862fcb0
181ce2a1082b28a7fdcb8c8c0c6e586e
521c9cb249d37225ae52e6d3e558aa5c


In [50]:
print(json.dumps(test_res["72b3b4cf06c0315e82e4be3799b326fa"], indent=4))

{
    "admin1": "Mugla",
    "month": 11,
    "weekday": 1,
    "indoor_outdoor": "indoor",
    "body_face_cloth": [
        {
            "age": 24,
            "gender": "F",
            "face": [
                236.2800312936306,
                84.17651210725307,
                317.2859801352024,
                171.4812659882009,
                0.9998173117637634
            ],
            "clothes": [
                {
                    "box": [
                        160,
                        154,
                        512,
                        385
                    ],
                    "upper_body_length:hip_length": 1,
                    "closure_type:pullover": 1,
                    "category": "upper_body_garment__sweater",
                    "product_color:yellow": 1,
                    "sleeve_style:straight": 1,
                    "sleeve_length:long": 1,
                    "neckline:off_shoulder": 1
                },
                {
           

In [19]:
id_ = 27
print(test_ids[id_])
grd = id_meta_map[test_ids[id_]]
print("occ is: ", grd["occasion"])
for each in grd["cloth_body_face"]:
    print("cat is: ", each["cloth"]["category"])
    print("x", (each["cloth"]["box"][0]+each["cloth"]["box"][2])/2)
    for k, v in each["cloth"].items():
        if k not in ["box", "category"]:
            print("attr", k, "val", v)

80841ab1c61ad87b49591098d3b38958
occ is:  conference
cat is:  full_body_garment__dress
x 176.0
attr lower_body_length:7_8 val 0.5515718460083008
attr sleeve_style:na val 0.5960307717323303
attr product_color:purple val 0.17167992889881134
attr neckline:round_neck val 0.34016111493110657
attr sleeve_length:sleeveless val 0.931877374649048
attr dress_skirt_shape:a_line val 0.9843226075172424


In [20]:
occ, cat, attrs = recover(attr_res[id_], cat_res[id_], cat_mask[id_], occ_res[id_], cat_attr_mask)
print("occ is: %s\ncat is: %s\n" %(occ, cat))
for attr in attrs:
    print(attr)

occ is: prom
cat is: full_body_garment__dress

dress_skirt_shape:a_line
lower_body_length:knee_length
neckline:v_neck
product_color:pink
sleeve_length:sleeveless
sleeve_style:na
